In [1]:
import numpy as np
import math
import cv2
from skimage.metrics import structural_similarity
import imutils
import os, sys
import statistics 
import os
from prettytable import PrettyTable
import fnmatch
from glob import glob
import time

In [2]:
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [3]:
def ssim(im1,im2):
    grayA = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    (score, diff) = structural_similarity(grayA, grayB, full=True)
    return score

In [4]:
def get_progress(progress):
    max_length=20
    block = int(round(max_length*progress))
    status=" Done..." if progress>=1 else " "
    return ('Progress:[' + '=' * block +
            ('>' if block < max_length else '') +
            ' ' * (max_length - block) +
            '] %.1f%%' % (progress * 100.)+status)

In [7]:
fold_ssim_mean=[]
fold_psnr_mean=[]
fold_ssim_stdev=[]
fold_psnr_stdev=[]

for fold_number in range(1,6):
    inc = 0
    files=[]
    preds=[]
    ground_truths=[]
    psnrs=[]
    ssims=[]
    start_dir = os.getcwd() + "./results/unet_256/test_latest/fold"+str(fold_number)
    patterns   = ["*.png", "*.jpg","*.jpeg"]
    for pattern in patterns:
        for dir,_,_ in sorted(os.walk(start_dir)):
            files.extend(glob(os.path.join(dir,pattern))) 
    print("Processing Fold #{}".format(fold_number))
    
    for file in files:
        if "fake_B" in file:
            preds.append(file)
        elif "real_B" in file:
            ground_truths.append(file)

    number_of_files=len(preds)
    
    for i in range(number_of_files):
        gt=ground_truths[i]
        pred=preds[i]
        gt_name=gt[gt.rfind("/")+1:]
        pred_name=pred[pred.rfind("/")+1:]
        gt_image = cv2.imread(gt)
        pred_image = cv2.imread(pred)
        p=psnr(gt_image,pred_image)
        s=ssim(gt_image,pred_image)
        inc = inc + 1
        sys.stderr.write('\r\033[K' + get_progress(inc/number_of_files))
        #sys.stderr.flush()
        psnrs.append(p)
        ssims.append(s)
           
    fold_psnr_mean.append(statistics.mean(psnrs))
    fold_ssim_mean.append(statistics.mean(ssims))
    fold_psnr_stdev.append(statistics.stdev(psnrs))
    fold_ssim_stdev.append(statistics.stdev(ssims))
    time.sleep(.1)
    sys.stderr.flush()
print("Calculation completed")

Processing Fold #1


Progress:[====================] 100.0% Done...

Processing Fold #2


Progress:[====================] 100.0% Done...

Processing Fold #3


...

Processing Fold #4


Progress:[====================] 100.0% Done...

Processing Fold #5


Progress:[====================] 100.0% Done...

Calculation completed


In [8]:
x = PrettyTable()
x.field_names = ["Algorithm name", "Fold#", "PSNR Fold Mean", "SSIM Fold Mean", "PSNR Fold Stdev", "SSIM Fold Stdev"]
y = PrettyTable()
y.field_names = ["Algorithm name", "PSNR Mean", "PSNR Stdev", "SSIM Mean", "SSIM Stdev"]

algorithm="EndoL2H"

for i in range(len(fold_psnr_mean)):
    x.add_row([algorithm, i+1, fold_psnr_mean[i],fold_ssim_mean[i], fold_psnr_stdev[i], fold_ssim_stdev[i]])

print(x)

if len(fold_psnr_mean)>1:
    y.add_row([algorithm, statistics.mean(fold_psnr_mean), statistics.stdev(fold_psnr_stdev), statistics.mean(fold_ssim_mean), statistics.stdev(fold_ssim_stdev)])
    print()
    print(y)

+----------------+-------+--------------------+--------------------+--------------------+---------------------+
| Algorithm name | Fold# |   PSNR Fold Mean   |   SSIM Fold Mean   |  PSNR Fold Stdev   |   SSIM Fold Stdev   |
+----------------+-------+--------------------+--------------------+--------------------+---------------------+
|    EndoL2H     |   1   | 33.79275371045731  | 0.7790007884475312 | 1.3445908121342998 |  0.0827830884611864 |
|    EndoL2H     |   2   | 32.62549169387905  | 0.7199665061187138 | 1.1351787126223385 | 0.10028190042772774 |
|    EndoL2H     |   3   | 32.943964730637724 | 0.7407974120855597 | 1.083992121997701  | 0.08894786990424274 |
|    EndoL2H     |   4   | 32.582642405605526 | 0.7113114089565795 | 1.1007161521776727 | 0.10321824165628292 |
|    EndoL2H     |   5   | 32.87387711353836  | 0.7304619223128992 | 1.232022178084014  | 0.09930372108879008 |
+----------------+-------+--------------------+--------------------+--------------------+---------------